In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [3]:
from huggingface_hub import login

login(token="YOUR-KEY-HUGGING-FACE")

#**Finetune**

In [4]:

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Thinking-2507-unsloth-bnb-4bit",
    max_seq_length = 1024,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-09-04 09:56:41.717433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756979802.109986      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756979802.221755      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [6]:

model = FastLanguageModel.get_peft_model(
    model,
        r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 32,  # Best to choose alpha = rank or rank*2
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)


Unsloth: Already have LoRA adapters! We shall skip this step.


In [7]:

import re

def remove_markdown(text: str) -> str:
    # Loại bỏ tô đậm **text**
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)

    # Loại bỏ bullet point bắt đầu bằng *, -, hoặc + và khoảng trắng sau đó
    text = re.sub(r'^\s*[\*\-\+]\s+', '', text, flags=re.MULTILINE)

    # Loại bỏ tiêu đề markdown như ###, ####
    text = re.sub(r'^#{1,6}\s*', '', text, flags=re.MULTILINE)

    # Thay thế nhiều dòng liên tục bằng dòng trống
    text = re.sub(r'\n+', '\n', text)

    # Loại bỏ khoảng trắng dư thừa ở đầu và cuối
    return text.strip()


In [8]:

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-thinking",
)


In [9]:
from datasets import load_dataset

dataset = load_dataset("thailevann/durationQA_track8_vlsp2025")

README.md:   0%|          | 0.00/607 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/246k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/221 [00:00<?, ? examples/s]

In [ ]:

from datasets import Dataset
from transformers import AutoTokenizer
import json
'''
# 1. Load dữ liệu từ JSONL thành list of dicts
path = '/kaggle/working/duration_reasoning_output.jsonl'
data = [json.loads(line) for line in open(path, 'r', encoding='utf-8')]

# 2. Chuyển thành Huggingface Dataset
dataset = Dataset.from_list(data)
# 4. Áp dụng hàm cho toàn bộ dataset
dataset = dataset.map(add_sorted_options_reason)
'''

# 5. Kiểm tra kết quả
#print(dataset['train'][0]['CoT'])
def generate_conversation(batch):
    conversations = []
    for i in range(len(batch["question"])):
        question  = batch["question"][i]
        reasoning = remove_markdown(batch["CoT"][i])
        options   = batch["options"][i]
        labels    = batch["labels"][i]
        context   = batch["context"][i]

        # Format lựa chọn
        options_text = "\n".join([f"{idx+1}. {opt}" for idx, opt in enumerate(options)])

        problem = (
                    f"{context}\n\n"
                    f"Câu hỏi: {question}\n"
                    f"Lựa chọn:\n{options_text}\n\n"
                    f"Hãy suy luận theo **Chain-of-Thought (CoT)** để tìm ra lựa chọn đúng.\n"
                    f"Viết phần suy luận vào thẻ <think>.  Sau đó, liệt kê nhãn tương ứng của từng lựa chọn trong thẻ <labels>.\n"
        )
        # Lời giải
        solution = (
            f"<think>\n{reasoning.strip()}\n</think>\n"
            f"<labels>{', '.join([label.strip().lower() for label in labels])}</labels>"
        )

        conversations.append([
            {"role": "user", "content": problem},
            {"role": "assistant", "content": solution},
        ])

    return {"conversations": conversations}


reasoning_conversations = tokenizer.apply_chat_template(
    dataset['train'].map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)


In [ ]:

from datasets import Dataset
from transformers import AutoTokenizer
import json
'''
# 1. Load dữ liệu từ JSONL thành list of dicts
path = '/kaggle/working/duration_reasoning_output.jsonl'
data = [json.loads(line) for line in open(path, 'r', encoding='utf-8')]

# 2. Chuyển thành Huggingface Dataset
dataset = Dataset.from_list(data)
dataset = dataset.map(add_sorted_options_reason)
'''

def nongenerate_conversation(batch):
    conversations = []
    for i in range(len(batch["question"])):
        question  = batch["question"][i]
        reasoning = remove_markdown(batch["CoT"][i])
        options   = batch["options"][i]
        labels    = batch["labels"][i]
        context   = batch["context"][i]

        # Format lựa chọn
        options_text = "\n".join([f"{idx+1}. {opt}" for idx, opt in enumerate(options)])
        problem = (
            f"{context}\n\n"
            f"Câu hỏi: {question}\n"
            f"Suy luận chi tiết cho từng lựa chọn:\n{reasoning.strip()}\n"
            f"Các lựa chọn:\n{options_text}\n\n"
            "Nhiệm vụ:\n"
            "1. Đọc phần suy luận của từng lựa chọn.\n"
            "2. Chỉ gán nhãn 1 cho phương án được tổng kết là hợp lý nhất.\n"
            "3. Tất cả các lựa chọn khác gán nhãn 0, ngay cả khi suy luận nói 'có thể hợp lý trong điều kiện đặc biệt'.\n"
            "4. Kết quả phải hoàn toàn nhất quán với phần tổng kết suy luận.\n"
            "5. Nếu dựa vào context và question mà không thể xác định option nào hợp lý nhất, gán 0 cho tất cả các option.\n"
            "6. Liệt kê nhãn tương ứng của từng lựa chọn theo đúng thứ tự, trong thẻ <labels>, ví dụ: <labels>1 0 0 0</labels>."
        )

        # Lời giải
        solution = (
            f"<labels>{', '.join([label.strip().lower() for label in labels])}</labels>"
        )

        conversations.append([
            {"role": "user", "content": problem},
            {"role": "assistant", "content": solution},
        ])

    return {"conversations": conversations}


nonreasoning_conversations = tokenizer.apply_chat_template(
    dataset['train'].map(nongenerate_conversation, batched = True)["conversations"],
    tokenize = False,
)


In [ ]:

import pandas as pd
data = pd.concat([
    pd.Series(reasoning_conversations),
    pd.Series(nonreasoning_conversations)

])
data.name = "text"

from datasets import Dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)


In [ ]:

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=combined_dataset,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=2,
        learning_rate=2e-5,
        warmup_ratio=0.03,
        logging_steps=10,
        save_strategy="epoch",
        #evaluation_strategy="epoch",
        save_total_limit=2,
        #load_best_model_at_end=True,
        fp16=True,  # hoặc bf16 nếu hỗ trợ
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)


In [ ]:

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)


In [ ]:

# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


In [ ]:
#tokenizer.decode(trainer.train_dataset[100]["input_ids"])

In [ ]:
#tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:

trainer_stats = trainer.train()
model.push_to_hub("thailevann/cot_refine") 
tokenizer.push_to_hub("thailevann/cot_refine") 
